In [ ]:
import torch

In [ ]:
print("Naziv GPU-a:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nema GPU-a")

In [ ]:
!pip install opencv-python
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import numpy as np
import itertools
import datetime
import cv2
import os
import io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import numpy as np
import itertools
import datetime
import cv2
import os
import io
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, labels, transform=None):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.image_paths = []
        self.image_labels = []
        self.classes = labels

        for label in labels:
            folder = os.path.join(root_dir, label)
            for filename in os.listdir(folder):
                if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    continue

                self.image_paths.append(os.path.join(folder, filename))
                self.image_labels.append(labels.index(label))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.image_labels[idx]

        image = cv2.imread(img_path, 0)  # grayscale
        if image is None:
            raise ValueError(f"Image at path '{img_path}' could not be loaded.")
        image = cv2.bilateralFilter(image, 2, 50, 50)
        image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
        image = cv2.resize(image, (224, 224))
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 1. loadaj dataset
full_dataset = BrainTumorDataset("/content/drive/My Drive/training", labels, transform=transform)
print("dataset size: ", len(full_dataset))

indices = list(range(len(full_dataset)))

# podjela dataseta u trening i validation(80%- 20%)
train_indices, val_indices = train_test_split(
    indices,
    test_size=0.2,
    stratify=full_dataset.image_labels,
    random_state=42
)

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)
print("training dataset size: ", len(train_dataset))
print("validation dataset size: ", len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
root_dir = "/content/drive/My Drive/training"
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

for label in classes:
    folder = os.path.join(root_dir, label)
    num_images = sum(
        fname.lower().endswith(('.jpg', '.jpeg', '.png')) for fname in os.listdir(folder)
    )
    print(f"{label}: {num_images} images")

glioma: 1321 images
meningioma: 1339 images
notumor: 1595 images
pituitary: 1457 images


In [ ]:
#VIZUALIZACIJA
data_iter = iter(train_loader)
images, batch_labels = next(data_iter)
#pretvaranje slike u numpy polje i normalizacija
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
images = (images.numpy().transpose((0, 2, 3, 1)) * std + mean).clip(0, 1)

#vizualizacija
num_images = len(images)
rows = int(np.ceil(num_images / 4))
fig, axes = plt.subplots(rows, 4, figsize=(15, 15))


for i, ax in enumerate(axes.flat):
    if i < num_images:
        ax.imshow(images[i])
        class_index = batch_labels[i].item()  # convert tensor to int
        ax.set_title(f'Label: {train_dataset.dataset.classes[class_index]}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=15, name='model', patience=7):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_val_loss = float('inf')
    tolerance = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        # === Training ===
        model.train()
        train_loss, correct_train, total_train = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        train_loss /= len(train_loader)
        train_acc = 100 * correct_train / total_train

        # === Validation ===
        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100 * correct_val / total_val

        # === Logging ===
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")
        print("-" * 60)

        # === Early Stopping ===
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'best_brain_tumor_{name}.pth')
            tolerance = 0
        else:
            tolerance += 1
            if tolerance >= patience:
                print(f"Early stopping after {epoch+1} epochs.")
                break

    return history

In [ ]:
model2 = models.resnet18(pretrained=True)

for param in model2.parameters():
    param.requires_grad = False

num_ftrs = model2.fc.in_features
model2.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 4)
)

model2 = model2.to(device)
criterion = nn.CrossEntropyLoss()


In [ ]:
for name, param in model2.named_parameters():
    if 'layer4' in name or 'layer3' in name or 'fc' in name:
        param.requires_grad = True


In [ ]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model2.parameters()), lr=1e-4)

In [ ]:
print("Starting training...")
history = train_model(model2, train_loader, val_loader, criterion, optimizer, num_epochs=5, name='resnet18_finetuned')



In [ ]:
model2 = model2.to(device)
model2.eval()

In [ ]:
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
test_dataset = BrainTumorDataset("/content/drive/My Drive/Testing", class_names, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print("test dataset size: ", len(test_dataset))


test dataset size:  1311


In [ ]:
def test_model(model, test_loader, num_images_to_show=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    all_preds = []
    all_labels = []
    all_images = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # ?
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_images.extend(images.cpu())
    test_acc = 100 * correct / total

    # matrica zabune
    cm = confusion_matrix(all_labels, all_preds)

    print(f"Test Accuracy: {test_acc:.2f}%\n")


    print("Classification Report:\n")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    #prikaz matrice zabune
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Matrica zabune')
    plt.xlabel('Predviđena klasa')
    plt.ylabel('Istinita klasa')
    plt.show()

In [ ]:
test_model(model2, test_loader)

In [ ]:
#obično testiranje pojedinih slika (ručno promijeniti path)

model2.eval()
image_path1="/content/drive/My Drive/Testing/meningioma/Te-me_0078.jpg"
image = Image.open(image_path1).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

input_tensor = transform(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = model2(input_tensor)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

print("Predicted class index:", predicted_class)


print("Predicted label:", class_names[predicted_class])

In [ ]:
!pip install grad-cam

from torchvision import transforms
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam import GradCAM, GuidedBackpropReLUModel
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
def run_gradcam(model, image_path, target_layer, class_names):
    # load i preprocess slike
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0)

    # prebacivanje na gpu
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    input_tensor = input_tensor.to(device)

    # kreiranje CAM objekta
    cam = GradCAM(model=model, target_layers=[target_layer])
    cam.batch_size = 1

    # predikcija
    input_tensor.requires_grad_()
    output = model(input_tensor)
    print("Raw output (logits):", output)

    # ovo mozes izbrisati
    probabilities = torch.nn.functional.softmax(output, dim=1)
    print("Probabilities:", probabilities)
    predicted_class = output.argmax(dim=1).item()

    # Runnaj Grad-CAM
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(predicted_class)])
    grayscale_cam = grayscale_cam[0, :]

    # Prepare original image for overlay?????
    rgb_image = image.resize((224, 224))
    rgb_np = np.array(rgb_image).astype(np.float32) / 255.0
    cam_image = show_cam_on_image(rgb_np, grayscale_cam, use_rgb=True)

    # Display
    plt.imshow(cam_image)
    plt.title(f"Predicted: {class_names[predicted_class]}")
    plt.axis('off')
    plt.show()


image_path = '/content/drive/My Drive/Testing/pituitary/Te-pi_0094.jpg'
target_layer = model2.layer3[-1]


run_gradcam(model2, image_path, target_layer, class_names)


In [ ]:
def run_gradcam_improved(model, image_path, target_layer, class_names_list, preprocess_fn):
    import torch, numpy as np
    from PIL import Image
    from pytorch_grad_cam import GradCAM
    from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
    from pytorch_grad_cam.utils.image import show_cam_on_image
    import matplotlib.pyplot as plt

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()

    original_param_grad_states = {name: p.requires_grad for name, p in model.named_parameters()}
    for p in model.parameters():
        p.requires_grad_(True)

    try:
        image_pil = Image.open(image_path).convert('RGB')
        #image_pil = image_pil.resize((1024, 1024))
    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
        return

    input_tensor = preprocess_fn(image_pil).unsqueeze(0).to(device)

    cam = GradCAM(model=model, target_layers=[target_layer])
    cam.batch_size = 1

    with torch.no_grad():
        output_logits = model(input_tensor)

    probabilities = torch.nn.functional.softmax(output_logits, dim=1)
    predicted_class_idx = probabilities.argmax(dim=1).item()
    predicted_class_name = class_names_list[predicted_class_idx]

    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(predicted_class_idx)])
    grayscale_cam = grayscale_cam[0, :]
    grayscale_cam = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min())
    #threshold = np.percentile(grayscale_cam, 95)
    #grayscale_cam[grayscale_cam < threshold] = 0

    rgb_image = image_pil.resize((224, 224))
    rgb_np = np.array(rgb_image).astype(np.float32) / 255.0

    cam_image = show_cam_on_image(
    rgb_np,
    grayscale_cam,
    use_rgb=True,
    colormap=cv2.COLORMAP_JET,
    image_weight=0.5
)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(image_pil)
    axs[0].set_title("Original")
    axs[0].axis('off')
    axs[1].imshow(cam_image)
    axs[1].set_title("Grad-CAM")
    axs[1].axis('off')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(7, 7))
    plt.imshow(cam_image)
    plt.title(f"Grad-CAM | Predicted: {predicted_class_name}")
    plt.axis('off')
    plt.show()

    for name, p in model.named_parameters():
        p.requires_grad_(original_param_grad_states.get(name, False))

    return grayscale_cam, predicted_class_idx


In [ ]:
#prikaz Grad-CAM
image_path = '/content/drive/My Drive/Testing/meningioma/Te-me_0135.jpg'
target_layer = model2.layer4[-1]
grayscale_cam, predicted_class_idx = run_gradcam_improved(
    model2,
    image_path,
    target_layer,
    class_names,
    preprocess
)

In [ ]:
def run_grad_cam_plus_plus_adapted(model, image_path, target_layer, class_names_list, preprocess_fn):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    original_param_grad_states = {name: p.requires_grad for name, p in model.named_parameters()}
    for p in model.parameters():
        p.requires_grad_(True)


    try:
        image_pil = Image.open(image_path).convert('RGB')
    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")

        for name, p in model.named_parameters():
            p.requires_grad_(original_param_grad_states.get(name, False))
        return

    input_tensor = preprocess_fn(image_pil).unsqueeze(0).to(device)
    cam_obj = GradCAMPlusPlus(model=model, target_layers=[target_layer])



    with torch.no_grad():
        output_logits = model(input_tensor)

    print("Raw output (logits):", output_logits)
    probabilities = torch.nn.functional.softmax(output_logits, dim=1)
    print("Probabilities:", probabilities)
    predicted_class_idx = probabilities.argmax(dim=1).item()
    predicted_class_name = class_names_list[predicted_class_idx]

    # Define targets for CAM
    targets_for_cam = [ClassifierOutputTarget(predicted_class_idx)]

    # Run Grad-CAM++
    grayscale_cam = cam_obj(input_tensor=input_tensor, targets=targets_for_cam)
    grayscale_cam = grayscale_cam[0, :] # Get the CAM for the first (and only) image

    grayscale_cam = (grayscale_cam - grayscale_cam.min()) / (grayscale_cam.max() - grayscale_cam.min())

    rgb_image_for_viz = image_pil.resize((224, 224))
    rgb_np_for_viz = np.array(rgb_image_for_viz).astype(np.float32) / 255.0

    cam_visualization = show_cam_on_image(rgb_np_for_viz, grayscale_cam, use_rgb=True)

    import matplotlib.pyplot as plt

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(image_pil)
    axs[0].set_title("Original")
    axs[0].axis('off')

    axs[1].imshow(cam_visualization)
    axs[1].set_title("Grad-CAM++")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()


    # Prikaz heatmape
    plt.figure(figsize=(7,7))
    plt.imshow(cam_visualization)
    plt.title(f"Grad-CAM++ | Predicted: {predicted_class_name}")
    plt.axis('off')
    plt.show()

    for name, p in model.named_parameters():

        p.requires_grad_(original_param_grad_states.get(name, False))
    return grayscale_cam, predicted_class_idx


target_conv_layer = model2.layer4[-1]
test_image_path = '/content/drive/My Drive/Testing/pituitary/Te-pi_0089.jpg' # mjenjaj ručno

if not os.path.exists(test_image_path):
    print(f"Kriva slika: {test_image_path}")
else:
    print("Running Grad-CAM++")
    grayscale_cam, class_index = run_grad_cam_plus_plus_adapted(
    model2, test_image_path, target_conv_layer, class_names, preprocess
)



In [ ]:

rgb_img = np.array(Image.open(image_path).convert('RGB')) / 255.0

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

input_tensor = transform(Image.open(image_path)).unsqueeze(0)
print(input_tensor.shape) # shape: [1, 3, 224, 224]
input_tensor = input_tensor.to('cuda')

In [ ]:
!pip install shap

In [ ]:
import shap
from torchvision import datasets, transforms

model2.eval()
notumor_dir = "/content/drive/My Drive/Testing/notumor"

notumor_dataset = datasets.ImageFolder(
    root="/content/drive/My Drive/Testing",
    transform=transform
)
notumor_label = class_names.index('notumor')

# Filtriraj samo za notumor
notumor_indices = [i for i, (_, label) in enumerate(notumor_dataset) if label == notumor_label]


In [ ]:
selected_indices = notumor_indices[:100]   #100  slika za backgound skup podataka
notumor_subset = Subset(notumor_dataset, selected_indices)

notumor_loader = DataLoader(notumor_subset, batch_size=50, shuffle=False)

background_images, _ = next(iter(notumor_loader))
background_images = background_images.to(device)


explainer = shap.GradientExplainer(model2, background_images)

In [ ]:
img_path = "/content/drive/My Drive/Testing/pituitary/Te-pi_0089.jpg"

pil_img     = Image.open(img_path).convert("RGB")
test_image  = transform(pil_img).unsqueeze(0).to(device)

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

#prikaz slike
def show_image(tensor, title=None):
    image = tensor.squeeze(0).cpu().detach().numpy().transpose(1, 2, 0)  # shape: HWC
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean  # de-normalize
    image = np.clip(image, 0, 1)

    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()
show_image(test_image)


In [ ]:
shap_values = explainer.shap_values(test_image)   #shap vrijednosti

In [ ]:
threshold = np.percentile(np.abs(shap_values), 95)    #5% najznacajnijih vrijednosti
shap_values = np.where(np.abs(shap_values) >= threshold, shap_values, 0)


In [ ]:

test_image_np = test_image.cpu().numpy().transpose(0, 2, 3, 1)


with torch.no_grad():
    output = model2(test_image)
    predicted_class = output.argmax(1).item()
shap_values_predicted_class = shap_values[:, :, :, :, predicted_class]
shap_values_predicted_class = shap_values_predicted_class.squeeze(0)
shap_values_for_plot = shap_values_predicted_class.transpose(1, 2, 0)

shap.image_plot([shap_values_for_plot], test_image_np[0])
